In [1]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [2]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))
                
from spinorama.graph_isobands import find_isobands
def transform_id(x):
    return x

In [3]:
p0 = (0,0)
p1 = (2,0)
p2 = (2,1)
p3 = (0,1)

x = [[0, 2], [0, 2]]
y = [[0, 1], [2, 1]]
z = [[3, 2], [2, 3.2]]

df = find_isobands(x, y, z, [0, 1.5, 2.5, 3], transform_id, transform_id)

g_df = alt.Chart(alt.Data(values=df['features'])
).mark_geoshape(
    stroke='red',
    strokeWidth=2,
    clip=True,
    cornerRadius=2
).encode(
    alt.Color(
        'properties.z_low:Q', 
        scale=alt.Scale(scheme='bluegreen'),
        title = "IsoBands")
).project('identity'
).properties(
    width=200,
    height=200
)

g_df

alt.Chart(...)

In [4]:
xr = np.linspace(-5, 5, 30)
yr = np.linspace(-5, 5, 30)
x, y = np.meshgrid(xr, yr)
z = x ** 2 + y ** 2
print(len(xr), len(yr), z.shape)

source = pd.DataFrame({'x': x.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

df = find_isobands(x, y, z, [1, 2, 4, 8, 16, 32], transform_id, transform_id)

g_source = alt.Chart(source).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q')

g_iso = alt.Chart(alt.Data(values=df['features'])
).mark_geoshape(
    stroke='red',
    strokeWidth=0,
    clip=True,
    cornerRadius=0
).encode(
    alt.Color(
        'properties.z_low:Q', 
        scale=alt.Scale(scheme='bluegreen'),
        title = "IsoBands")
).project('identity'
)

#g_source+g_iso    
g_iso


30 30 (30, 30)


alt.Chart(...)

In [5]:
from spinorama.load import graph_melt
import flammkuchen as fl
from spinorama.graph_contour import compute_contour
from spinorama.graph import contour_params_default, graph_contour_common

df_all = fl.load('../cache.parse_all_speakers.h5')

In [6]:
df = df_all['Genelec 8341A']['ASR']['asr']
print(df.keys())

dict_keys(['CEA2034', 'CEA2034_unmelted', 'Directivity Index', 'Directivity Index_unmelted', 'Early Reflections', 'Early Reflections_unmelted', 'Estimated In-Room Response', 'Estimated In-Room Response_unmelted', 'Horizontal Reflections', 'Horizontal Reflections_unmelted', 'SPL Horizontal', 'SPL Horizontal_unmelted', 'SPL Vertical', 'SPL Vertical_unmelted', 'Vertical Reflections', 'Vertical Reflections_unmelted', 'CEA2034_original_mean'])


In [7]:
dfu = df['SPL Vertical_unmelted']
af, am, az = compute_contour(dfu.loc[dfu.Freq>100])
print(len(af[0]), len(am[:,0]), az.shape )
print(af[0][0:3])
print(am[:,0])

154 36 (36, 154)
[20.         20.92366889 21.88999599]
[-170 -160 -150 -140 -130 -120 -110 -100  -90  -80  -70  -60  -50  -40
  -30  -20  -10    0   10   20   30   40   50   60   70   80   90  100
  110  120  130  140  150  160  170  180]


In [8]:
graph_width = 1024
graph_height = 720

def transform_id(x):
    return x

def transform_log(x):
    return 6*np.log10(x)*graph_width/800

def transform_radian(x):
    return 1.325*x/180*math.pi*graph_height/360

bands = [-72, -18, -15, -12, -9, -6, -3, +3]
df_iso = find_isobands(af, am, az.T, bands, transform_log, transform_radian)

color_legend = ['[{0}, {1}]'.format(bands[i], bands[i+1]) for i in range(0, len(bands)-1)] + ['>{0}'.format(bands[-1])]
color_range = ['black', 'blue', 'steelblue', 'green', 'yellow', 'orange', 'red', 'white']

print(color_legend)

isobands = alt.Chart(alt.Data(values=df_iso['features'])
).mark_geoshape(
    stroke='red',
    strokeWidth=0
).encode(
    # alt.Color('properties.z_low:Q', scale=alt.Scale(scheme='inferno'), title = "IsoBands")
    alt.Color('properties.z_low:O', 
              scale=alt.Scale(zero=False, domain=bands, range=color_range), 
              legend=alt.Legend(orient='bottom', title='Relative dB SPL'))
).project('identity'
).properties(
    width=graph_width,
    height=graph_height
)

['[-72, -18]', '[-18, -15]', '[-15, -12]', '[-12, -9]', '[-9, -6]', '[-6, -3]', '[-3, 3]', '>3']


In [9]:
empty = pd.DataFrame({'Freq', 'Angle'})
yTicks = [-180+30*i for i in range(0,13)]
print(yTicks)
legends = alt.Chart(empty).mark_point(clip=True).encode(
    x=alt.X('x:Q', scale=alt.Scale(type='log', nice=False, domain=[20, 20000]), title='Frequency (Hz)'),
    y=alt.Y('y:Q', scale=alt.Scale(nice=False, domain=[-180, 180]), axis=alt.Axis(format='.0d', values=yTicks, title='Angle')), 
).properties(
    width=graph_width,
    height=graph_height
)
legends

[-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]


alt.Chart(...)

In [10]:
isobands + legends

alt.LayerChart(...)

In [11]:
len(af)

36

In [12]:
am[:,0]

array([-170, -160, -150, -140, -130, -120, -110, -100,  -90,  -80,  -70,
        -60,  -50,  -40,  -30,  -20,  -10,    0,   10,   20,   30,   40,
         50,   60,   70,   80,   90,  100,  110,  120,  130,  140,  150,
        160,  170,  180])

In [13]:
az

array([[-1.03100e-01, -1.74700e-01, -1.78400e-01, ..., -4.49856e+01,
        -4.15761e+01, -4.01879e+01],
       [-1.29200e-01, -1.97900e-01, -2.01100e-01, ..., -3.87289e+01,
        -3.78227e+01, -3.93127e+01],
       [-1.53600e-01, -2.19800e-01, -2.25100e-01, ..., -4.31335e+01,
        -4.11431e+01, -4.14705e+01],
       ...,
       [ 1.45000e-02, -6.18000e-02, -7.84000e-02, ..., -5.59338e+01,
        -5.25955e+01, -4.24322e+01],
       [-3.24000e-02, -1.08500e-01, -1.19800e-01, ..., -4.86264e+01,
        -5.49216e+01, -4.20474e+01],
       [-7.16000e-02, -1.45900e-01, -1.52700e-01, ..., -4.89536e+01,
        -4.33893e+01, -4.17354e+01]])

In [14]:
def project(af, am):
    af2 = af
    am2 = am
    for i in range(0, len(af)):
        freqs = af[i]
        angles = am[:, i]
        af2[i] = [math.fabs(freq)*math.cos(angle) for freq, angle in zip(freqs, angles)]
        am2[i] = [math.fabs(freq)*math.sin(angle) for freq, angle in zip(freqs, angles)]
    return af2, am2

# af2, am2 = project(af, am) 

In [15]:
f = open('/tmp/example.geo.json', 'w')

In [16]:
import json
f.write(json.dumps(df_iso))

2988959

In [17]:
f.close()